## Testing of maintenance rates

From experimental data we have produced a set of maintenance rate estimates for each oxygen concentration and each possible path through the ETS of Av. While assuming the lowest maintenance rate would be biologically relevant we will test each maintenance rate with an experimentally derived substrate uptake rate and determine how accurate it can predict growth. Data is from Khula and Oelze 1988 and was extracted using [WebplotDigitizer](https://automeris.io/WebPlotDigitizer).

In [2]:
import cobra.test
import pandas as pd
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit
import numpy as np
import seaborn as sns
import os
import math
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model

In [6]:
#Load data from Fig 4 of Khula and Oelze 

Fig4_data = pd.read_csv("../Data/Experimental_Data/Khula_Oelze_1988_Figure_4.csv")
Fig4_data.head()

,Oxygen_conc,Growth_rate,Sucrose_uptake_p,Sucrose_uptake_gcdw
0,192,0.05,11.63,8.81
1,192,0.07,13.06,9.89
2,192,0.10,14.85,11.25
3,192,0.16,16.07,12.18
4,192,0.17,16.48,12.48


In [19]:
#load ATPM predictions
ATPM_pred = pd.read_csv("../Data/Maintenance_rates/Maintenance_rates_pred.csv", index_col=0)
ATPM_pred.rename({'O2_concentration': 'Oxygen_conc', 'ATPM_bound_value': 'ATPM_pred'}, axis=1, inplace=True) 
ATPM_pred.head()


,ETS_path,Oxygen_conc,group,ATPM_pred
0,NII_BD_F,12,NII_BD_F_12,16.258129
1,NII_BD_F,48,NII_BD_F_48,78.789395
2,NII_BD_F,108,NII_BD_F_108,110.805403
3,NII_BD_F,144,NII_BD_F_144,125.062531
4,NII_BD_F,192,NII_BD_F_192,143.071536


In [20]:
#Merge ATPM predictions and experimental data 

ATPM_pred_merge = pd.merge(left=ATPM_pred, right=Fig4_data, left_on='Oxygen_conc', right_on='Oxygen_conc')
ATPM_pred_merge

,ETS_path,Oxygen_conc,group,ATPM_pred,Growth_rate,Sucrose_uptake_p,Sucrose_uptake_gcdw
0,NII_BD_F,12,NII_BD_F_12,16.258129,0.06,2.65,2.01
1,NII_BD_F,12,NII_BD_F_12,16.258129,0.07,2.96,2.24
2,NII_BD_F,12,NII_BD_F_12,16.258129,0.10,2.96,2.24
3,NII_BD_F,12,NII_BD_F_12,16.258129,0.12,3.16,2.40
4,NII_BD_F,12,NII_BD_F_12,16.258129,0.18,5.00,3.79
...,...,...,...,...,...,...,...
283,NI_CO_R,192,NI_CO_R_192,446.723362,0.07,13.06,9.89
284,NI_CO_R,192,NI_CO_R_192,446.723362,0.10,14.85,11.25
285,NI_CO_R,192,NI_CO_R_192,446.723362,0.16,16.07,12.18
286,NI_CO_R,192,NI_CO_R_192,446.723362,0.17,16.48,12.48


In [ ]:
#reaction ids not in pathway then "name" of pathway
NII_BD_F = ["NADH6", "CYOO2pp", "RNF", "NII_BD_F"]
NII_CO_F = ["NADH6", "CYTBDpp", "RNF", "NII_CO_F"]
NII_BD_R = ["NADH6", "CYOO2pp", "FIX", "NII_BD_R"]
NII_CO_R = ["NADH6", "CYTBDpp", "FIX", "NII_CO_R"]
NI_BD_F = ["NADH5", "CYOO2pp", "RNF", "NI_BD_F"]
NI_CO_F = ["NADH5", "CYTBDpp", "RNF", "NI_CO_F"] 
NI_BD_R = ["NADH5", "CYOO2pp", "FIX", "NI_BD_R"]
NI_CO_R= ["NADH5", "CYTBDpp", "FIX", "NI_CO_R"]

all_paths = [NII_BD_F,
                NII_CO_F, 
                NII_BD_R,
                NII_CO_R,
                NI_BD_F,
                NI_CO_F,
                NI_BD_R,
                NI_CO_R]

In [ ]:
## code adapated from midnighter on github https://github.com/opencobra/cobrapy/issues/771
def metabolite_flux_balance(metabolite, solution):
    """
    Return a vector of reaction fluxes scaled by the stoichiometric coefficient.

    Parameters
    ----------
    metabolite : cobra.Metabolite
        The metabolite whose fluxes are to be investigated.
    solution : cobra.Solution
        Solution with flux values.

    Returns
    -------
    pandas.Series
        A vector with fluxes of reactions that consume or produce the given
        metabolite scaled by the corresponding stoichiometric coefficients. The
        reaction identifiers are given by the index.
    """
    rxn_ids = list()
    rxn_sub = list()
    adj_flux = list()
    for rxn in metabolite.reactions:
        coef = rxn.get_coefficient(metabolite)
        rxn_ids.append(rxn.id)
        rxn_sub.append(rxn.subsystem)
        adj_flux.append(coef * solution.fluxes[rxn.id])
    return pd.DataFrame({'Rxn_id': rxn_ids, 'Subsystem': rxn_sub, 'Adjusted_Flux': adj_flux})



#Two functions that will pull the total ATP produced and the ratio of ATPM/Total ATP from the above function
def ATP_total(df):
    positive_num = df[(df.Adjusted_Flux > 0)]
    ATP_total = df['Adjusted_Flux'].sum()
    return ATP_total

def ATP_ratio(df, ATP_total)
    ATPM = df.loc[df['Rxn_id'] == 'ATPM', 'Adjusted_Flux']]
    return ATPM
    

In [ ]:
def growth_rate_prediction(model_path, ):
    suc_uptake = ATPM_pred_merge['Sucrose_uptake_gcdw'].tolist()
    ATPM_flux = ATPM_pred_merge['ATPM_pred'].tolist()
    Growth_rate_pred =[]
    Oxygen_consumption_pred = [] 
    ATP_production = []
    ATPM_ATP_prect = []


    for (j, k) in zip(suc_uptake, ATPM_flux): 
        model = cobra.io.load_json_model(model_path)
        
        #Make sure glucose is set to zero
        model.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
        model.reactions.get_by_id("EX_glc__D_e").upper_bound = 0
        
        # Make sure glucose is set to zero
        model.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
        model.reactions.get_by_id("EX_glc__D_e").upper_bound = 0

        # Make sure model is diazotrophic
        model.reactions.get_by_id("EX_nh4_e").lower_bound = 0
        model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

        # set the flux to zero of corresponding reactions in pathway of interest
        model.reactions.get_by_id(rxn_path[0]).upper_bound = 0
        model.reactions.get_by_id(rxn_path[0]).lower_bound = 0

        model.reactions.get_by_id(rxn_path[1]).upper_bound = 0
        model.reactions.get_by_id(rxn_path[1]).lower_bound = 0

        model.reactions.get_by_id(rxn_path[2]).upper_bound = 0
        model.reactions.get_by_id(rxn_path[2]).lower_bound = 0

        # Set sucrose uptake to maintenance coefficient
        model.reactions.get_by_id("EX_sucr_e").lower_bound = -j
        model.reactions.get_by_id("EX_sucr_e").upper_bound = -j
 
        #Set ATPM to theorectical derived
        model.reactions.get_by_id("ATPM").upper_bound = 1000
        model.reactions.get_by_id("ATPM").lower_bound = k
    
        #solve the model

        solution = model.optimize()
    
        Growth_rate_pred.append(solution.objective_value)
        Oxygen_consumption_pred.append(solution.fluxes["O2tpp"])
        ATP_production.append(solution.fluxes["O2tpp"])
        Oxygen_consumption_pred.append(solution.fluxes["O2tpp"])    

    print(len(Oxygen_consumption_bd))
        
    
    Fig4_data['Growth_rate_exp_bd'] = Growth_rate_thr_bd
    Fig4_data['Oxygen_consumption_bd'] = Oxygen_consumption_bd    

    Fig4_data.to_csv("Predicted_maintence_rates_bd.csv")


    Fig4_data.head()